In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


In [95]:
# import something
master_df = pd.read_csv('../datasets/MASTER_DF.csv')
master_df.head()

,entry,product,reacts,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,1.14.99.46,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c6372b0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,3.5.1.110,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c637300>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.467093,0,0,...,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,1.14.99.46,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637210>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,1.17.99.4,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637620>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,1.3.1.1,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637670>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115


In [96]:
feature_df = master_df.iloc[:,7:24].copy()
feature_df.head()

,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,0.421512,1,0,0,0,0,0,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,0.467093,0,0,1,0,0,0,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,0.421512,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,0.414081,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,0.420926,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115


In [97]:
features = np.array(feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                    test_size=0.20, random_state=42)

In [11]:
model_1 = linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1)
model_1.fit(feature_train, np.ravel(reaction_train))

predictions = model_1.predict(feature_test) # change me to the data you want to predict based on 

score = model_1.score(feature_test, reaction_test)
decision = model_1.decision_function(feature_test)
params = model_1.get_params()
pred_log = model_1.predict_log_proba(feature_test)

pred = model_1.predict_proba(feature_test)

score, pred, decision, model_1.classes_, model_1.coef_

(0.9429882044560943, array([[0.93125186, 0.06874814],
        [0.71840718, 0.28159282],
        [0.98420052, 0.01579948],
        ...,
        [0.00606569, 0.99393431],
        [0.75956797, 0.24043203],
        [0.87533957, 0.12466043]]), array([-2.60608006, -0.93657437, -4.1318529 , ...,  5.09902319,
        -1.15031239, -1.94901841]), array([0., 1.]), array([[ 2.54706963e+01,  9.60277966e-03, -5.56434147e-02,
         -1.74946565e-01,  8.07018790e-01, -8.10073115e-01,
          0.00000000e+00,  0.00000000e+00, -4.73509960e-02,
         -5.86693377e-02, -1.96590660e-01, -2.29119563e-02,
         -1.04378788e-01,  5.84576921e-02, -1.56021011e-02,
         -1.06047143e-01,  7.47233475e-03]]))

In [12]:
decision_distances = pd.Series(decision)
decision_distances.describe()

count    1526.000000
mean       -1.490963
std         4.049824
min       -13.260221
25%        -3.736288
50%        -2.612336
75%        -1.242644
max        12.808593
dtype: float64

In [7]:
prediction_values = pd.DataFrame(pred)
model_descriptive_df = pd.DataFrame()
model_descriptive_df['0']=prediction_values[0]
model_descriptive_df['1']=prediction_values[1]
model_descriptive_df['decision_distance']=decision_distances
model_descriptive_df

,0,1,decision_distance
0,0.909955,0.090045,-2.313086
1,0.962006,0.037994,-3.231599
2,0.959105,0.040895,-3.154992
3,0.880714,0.119286,-1.999211
4,0.972866,0.027134,-3.579459
5,0.975632,0.024368,-3.689809
6,0.837659,0.162341,-1.640911
7,0.955413,0.044587,-3.064703
8,0.981334,0.018666,-3.962195
9,0.067622,0.932378,2.623801


In [8]:
updated = predictions.tolist()
confusion_matrix = confusion_matrix(reaction_test, updated)
print(confusion_matrix)
# upper left and lower right are correct: 1427
# lower left and upper right are incorrect : 99 

[[1176    7]
 [  94  249]]


In [9]:
print(classification_report(reaction_test, predictions))

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1183
         1.0       0.97      0.73      0.83       343

   micro avg       0.93      0.93      0.93      1526
   macro avg       0.95      0.86      0.90      1526
weighted avg       0.94      0.93      0.93      1526



In [14]:
model_2 = linear_model.LogisticRegressionCV(solver='liblinear', penalty='l1', random_state=1, cv=10)
model_2.fit(feature_train, np.ravel(reaction_train))
predictions2 = model_2.predict(feature_test)
score2 = model_2.score(feature_test, reaction_test)
decision2 = model_2.decision_function(feature_test)
params2 = model_2.get_params()
pred_log2 = model_2.predict_log_proba(feature_test)
pred2 = model_2.predict_proba(feature_test)

score2, pred2, #decision2, model_2.classes_, model_2.coef_

(0.9436435124508519, array([[0.93313034, 0.06686966],
        [0.71411113, 0.28588887],
        [0.98480214, 0.01519786],
        ...,
        [0.00532237, 0.99467763],
        [0.75659365, 0.24340635],
        [0.87789801, 0.12210199]]))

plan: ridge regression, drop off the least important features and get the AUC ROC curve-> store, drop the last
RFE again? (it didn't show anything...)


______________
# linear regression on default data

logreg1=linear_model.LogisticRegression(random_state=1)        
logreg1.fit(feature_train, np.ravel(reaction_train)) #fit linear model, and shape default array for regression
score = logreg1.score(feature_test, reaction_test)
decision = logreg1.decision_function(feature_test)
params = logreg1.get_params()
pred_log = logreg1.predict_log_proba(feature_test)
pred = logreg1.predict_proba(feature_test)

score, pred, logreg1.classes_, logreg1.coef_ #np.ndarray.shape(pred)

#print('B0, B1: ',logreg.intercept_, logreg.coef_[0])

predictions1 = logreg1.predict(feature_test)
confusion_matrix = confusion_matrix(reaction_test, predictions1)
print(confusion_matrix)
# upper left and lower right are correct: 1400 for the first go
# lower left and upper right are incorrect : 111 for the first go

print(classification_report(reaction_test, predictions1))

logit_roc_auc = roc_auc_score(reaction_test, logreg1.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg1.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, predictions1, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg5 = linear_model.LogisticRegression(penalty='l1', random_state=1)
logreg5.fit(feature_train, np.ravel(reaction_train))
predictions5 = logreg5.predict(feature_test)
score = logreg5.score(feature_test, reaction_test)
decision = logreg5.decision_function(feature_test)
params = logreg5.get_params()
pred_log = logreg5.predict_log_proba(feature_test)
pred = logreg5.predict_proba(feature_test)

score, pred, logreg5.classes_, logreg5.coef_

print(classification_report(reaction_test, predictions5))

logit_roc_auc = roc_auc_score(reaction_test, logreg5.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg5.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [16]:
from sklearn.feature_selection import RFE



In [98]:

rfe_results = pd.DataFrame(columns=['score']) 

rank_array = np.zeros((17,18))

for i in range(1,17):
    logreg2=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

    rfe = RFE(logreg2, i)
    rfe = rfe.fit(feature_train, np.ravel(reaction_train))
    score = rfe.score(feature_test, reaction_test)

    rfe_results.loc[i, 'score'] = score
    for j in range(0,17):  
        rank_array[i, j] = rfe.ranking_[j]
        

In [99]:
rank_df = pd.DataFrame(rank_array, columns=['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW', 'drop'])
#rfe_results
rank_df.drop(rank_df.index[0], inplace=True)
rank_df.drop(['drop'], axis=1, inplace=True)
rank_df

,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
1,1.0,16.0,10.0,6.0,3.0,2.0,15.0,14.0,17.0,12.0,8.0,9.0,11.0,5.0,4.0,7.0,13.0
2,1.0,15.0,9.0,5.0,2.0,1.0,14.0,13.0,16.0,11.0,7.0,8.0,10.0,4.0,3.0,6.0,12.0
3,1.0,14.0,8.0,4.0,1.0,1.0,13.0,12.0,15.0,10.0,6.0,7.0,9.0,3.0,2.0,5.0,11.0
4,1.0,13.0,7.0,3.0,1.0,1.0,12.0,11.0,14.0,9.0,5.0,6.0,8.0,2.0,1.0,4.0,10.0
5,1.0,12.0,6.0,2.0,1.0,1.0,11.0,10.0,13.0,8.0,4.0,5.0,7.0,1.0,1.0,3.0,9.0
6,1.0,11.0,5.0,1.0,1.0,1.0,10.0,9.0,12.0,7.0,3.0,4.0,6.0,1.0,1.0,2.0,8.0
7,1.0,10.0,4.0,1.0,1.0,1.0,9.0,8.0,11.0,6.0,2.0,3.0,5.0,1.0,1.0,1.0,7.0
8,1.0,9.0,3.0,1.0,1.0,1.0,8.0,7.0,10.0,5.0,1.0,2.0,4.0,1.0,1.0,1.0,6.0
9,1.0,8.0,2.0,1.0,1.0,1.0,7.0,6.0,9.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,5.0
10,1.0,7.0,1.0,1.0,1.0,1.0,6.0,5.0,8.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0


In [100]:
rank_df.sum(axis=0)
# lose MW, n_X, 

dist               16.0
enzyme_class_1    136.0
enzyme_class_2     61.0
enzyme_class_3     31.0
enzyme_class_4     19.0
enzyme_class_5     17.0
enzyme_class_6    121.0
enzyme_class_7    107.0
n_C               152.0
n_H                82.0
n_O                44.0
n_N                52.0
n_P                71.0
n_S                26.0
n_X                22.0
DoU                37.0
MW                 94.0
dtype: float64

In [101]:
rfe_results

,score
1,0.936435
2,0.936435
3,0.937746
4,0.931848
5,0.933159
6,0.931848
7,0.920052
8,0.921363
9,0.918087
10,0.918742


In [ ]:
logit_roc_auc = roc_auc_score(reaction_test, logreg1.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg1.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [39]:
feature_df.columns

Index(['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW'],
      dtype='object')

In [73]:
non_enzyme_feature_df = feature_df.copy()
non_enzyme_feature_df.drop(['enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7'], axis=1, inplace=True)
non_enzyme_feature_df



,dist,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,0.421512,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,0.467093,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,0.421512,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,0.414081,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,0.420926,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
5,0.446031,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
6,0.421512,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
7,0.439218,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
8,0.414081,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
9,0.420926,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088


In [91]:
features = np.array(non_enzyme_feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                  test_size=0.20, random_state=42)

In [92]:
non_enzyme_rfe_results = pd.DataFrame(columns=['score']) 

non_enzyme_rank_array = np.zeros((11,11))

for i in range(1,11):
    logreg2=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

    rfe = RFE(logreg2, i)
    rfe = rfe.fit(feature_train, np.ravel(reaction_train))
    score = rfe.score(feature_test, reaction_test)

    non_enzyme_rfe_results.loc[i, 'score'] = score
    for j in range(0,10):  
        non_enzyme_rank_array[i, j] = rfe.ranking_[j]
        

In [93]:
non_enzyme_rank_df = pd.DataFrame(non_enzyme_rank_array, columns=['dist',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW', 'drop'])
#rfe_results
non_enzyme_rank_df.drop(non_enzyme_rank_df.index[0], inplace=True)
non_enzyme_rank_df.drop(['drop'], axis=1, inplace=True)
non_enzyme_rank_df

,dist,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
1,1.0,10.0,8.0,6.0,5.0,7.0,3.0,2.0,4.0,9.0
2,1.0,9.0,7.0,5.0,4.0,6.0,2.0,1.0,3.0,8.0
3,1.0,8.0,6.0,4.0,3.0,5.0,1.0,1.0,2.0,7.0
4,1.0,7.0,5.0,3.0,2.0,4.0,1.0,1.0,1.0,6.0
5,1.0,6.0,4.0,2.0,1.0,3.0,1.0,1.0,1.0,5.0
6,1.0,5.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0
7,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
8,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
9,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [94]:
non_enzyme_rank_df.sum(axis=0)
# ditch n_N, MW, n_H, n_P

dist    10.0
n_C     55.0
n_H     38.0
n_O     25.0
n_N     20.0
n_P     31.0
n_S     13.0
n_X     11.0
DoU     16.0
MW      46.0
dtype: float64

In [86]:
features = list(master_df['dist']) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                  test_size=0.20, random_state=42)
a = len(feature_train)
b = len(feature_test)
feature_train = np.reshape(feature_train, (a, 1))
feature_test = np.reshape(feature_test, (b,1))

In [89]:

dist_only_model=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

dist_only_model.fit(feature_train, np.ravel(reaction_train))

predictions = dist_only_model.predict(feature_test) # change me to the data you want to predict based on 

score = dist_only_model.score(feature_test, reaction_test)
decision = dist_only_model.decision_function(feature_test)
params = dist_only_model.get_params()
pred_log = dist_only_model.predict_log_proba(feature_test)

pred = dist_only_model.predict_proba(feature_test)

score, pred, decision, dist_only_model.classes_, dist_only_model.coef_
        

(0.936435124508519, array([[0.90331985, 0.09668015],
        [0.64464838, 0.35535162],
        [0.777695  , 0.222305  ],
        ...,
        [0.0098376 , 0.9901624 ],
        [0.51115784, 0.48884216],
        [0.70442001, 0.29557999]]), array([-2.23466854, -0.59559723, -1.25228412, ...,  4.61165718,
        -0.04463876, -0.8684353 ]), array([0., 1.]), array([[17.87916795]]))

kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(features, reactions)

int_reactions = [int(i) for i in reactions]


for train_index, test_index in kf.split(features, reactions):
    #print("TRAIN:", train_index, "TEST:", test_index)
    feature_train, feature_test = features[train_index], features[test_index]
    reaction_train, reaction_test = np.array(int_reactions)[train_index], np.array(int_reactions)[test_index]

    reg = linear_model.LogisticRegression().fit(feature_train, reaction_train)
    y_pred = reg.predict(feature_test)

 

print(classification_report(reaction_test, y_pred))

logit_roc_auc = roc_auc_score(reaction_test, reg.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, reg.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, y_pred, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg3 = linear_model.LogisticRegression(solver='sag', max_iter=100000)
logreg3.fit(feature_train, np.ravel(reaction_train))
predictions3 = logreg3.predict(feature_test)

confusion_matrix = confusion_matrix(reaction_test, predictions3)
print(confusion_matrix)

print(classification_report(reaction_test, predictions3))

logit_roc_auc = roc_auc_score(reaction_test, logreg3.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg3.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

logreg4 = linear_model.LogisticRegression(solver='lbfgs', max_iter=10000)
logreg4.fit(feature_train, np.ravel(reaction_train))
predictions4 = logreg4.predict(feature_test)

logit_roc_auc = roc_auc_score(reaction_test, logreg4.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg4.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

print(classification_report(reaction_test, predictions4))